In [1]:
import json
import pandas as pd
import numpy as np
import folium
from folium.plugins import HeatMap

In [5]:
# read in data
df = pd.read_json('data/twitter/output_file_weekend.txt', lines = True)

# MAP

In [6]:
#fix coordinates field
#fill None with 0
filled = df['coordinates'].fillna(0)
#replace old column
df["coordinates"] = filled
#mask
coordinates = df[df['coordinates'] != 0]

#keep only needed vars;
coordinates = coordinates[["text","coordinates", "created_at", "lang", "possibly_sensitive", "id_str"]].reset_index()

#create separate lon and lat columns
coordinates_pd = pd.DataFrame(list(np.array(coordinates["coordinates"])))
coordinates_pd.rename(columns={'coordinates': 'coordinate_point'}, inplace=True)
df_coordinates = pd.concat([coordinates, coordinates_pd], axis=1, join='inner')
df_coordinates[['lon', 'lat']] = pd.DataFrame(df_coordinates['coordinate_point'].tolist())


def generateBaseMap(default_location=[29.4167, -98.5], default_zoom_start=12):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start)
    return base_map

base_map = generateBaseMap()


df_coordinates["count"] = 1
HeatMap(data=df_coordinates[['lat', 'lon', 'count']].groupby(['lat', 'lon']).sum().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(base_map)

base_map